In [346]:
import pandas as pd
import numpy as np

In [347]:
df = pd.read_csv('../data/diabetes.csv')

In [348]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [349]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [350]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [351]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

In [352]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 0)

In [353]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [354]:
model = Sequential()

model.add(Dense(32, input_dim = 8, activation = 'relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [355]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data = (X_test, y_test))
model.summary()

Epoch 1/100
20/20 [==============================] - 1s 11ms/step - loss: 0.6353 - accuracy: 0.6889 - val_loss: 0.5940 - val_accuracy: 0.7532
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5996 - accuracy: 0.7427 - val_loss: 0.5611 - val_accuracy: 0.7987
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5718 - accuracy: 0.7410 - val_loss: 0.5318 - val_accuracy: 0.7987
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5467 - accuracy: 0.7443 - val_loss: 0.5076 - val_accuracy: 0.7922
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5278 - accuracy: 0.7492 - val_loss: 0.4898 - val_accuracy: 0.8052
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5123 - accuracy: 0.7606 - val_loss: 0.4740 - val_accuracy: 0.7922
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4995 - accuracy: 0.7655 - val_loss: 0.4636 - val_accuracy: 0.7922
Epoch 8/100


In [356]:
import keras_tuner as kt

In [357]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32, input_dim = 8, activation = 'relu'))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop']), loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

In [358]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5
)

Reloading Tuner from .\untitled_project\tuner0.json


In [359]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

In [360]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'sgd'}

In [361]:
model = tuner.get_best_models(num_models=1)[0]

In [362]:
model.fit(X_train, y_train, validation_data = (X_test, y_test))

20/20 [==============================] - 0s 9ms/step - loss: 0.6485 - accuracy: 0.6726 - val_loss: 0.6243 - val_accuracy: 0.7532


In [363]:
def build_model2(hp):
    model = Sequential()

    units = hp.Int(8, 128, step=8)

    model.add(Dense(units, activation = 'relu', input_dim = 8))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

In [364]:
tuner2 = kt.RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=5
)

Reloading Tuner from .\untitled_project\tuner0.json


In [365]:
tuner2.search(X_train, y_train, validation_data = (X_test, y_test), epochs = 5)

In [366]:
tuner2.get_best_hyperparameters()[0].values

{'optimizer': 'sgd'}

In [368]:
model = tuner.get_best_models(num_models=1)[0]